# Assignment 2:  Motif finding


## Part 1:  Motif discovery using Gibbs sampling

In class we described the Gibbs sampling approach to motif finding.  Implement the algorithm and apply it to the sequences provided as the fasta file associated with this assignment, available via Canvas.  Run your algorithm and experiment with different motif lengths and other parameters of the algorithm.  Do you obtain consistent results across different runs of your code?  For scoring a motif use relative entropy.
The data you were given is taken from ChIP-seq experiments for the transcription factor USF1.  Compare the motif you found with the motif provided in the [CIS-BP database](http://cisbp.ccbr.utoronto.ca/) and comment on the similarity in terms of the pattern of nucleotides.  What is the relative entropy score of the final motif you obtained?

Note that the provided file contains a large number of sequences, which can make it difficult to test your code.
When developing your code, test it first on a toy dataset that you create by embedding a motif in a small set of short sequences.  Demonstrate that your algorithm indeed works on this small dataset.  We suggest to then evaluate your code on a sample of the provided sequences.

The motif found by your Gibbs sampler should be returned as a list-of-lists or two dimensional NumPy array, where each element in the list is a list of length four that corresponds to a position in the motif, and is a probability distribution over the nucleotides in that position, providing the probabilities for A, C, G, and T, respectively.

For example, the list 
```Python
[[0.5, 0, 0, 0.5], [0, 0.5,0.5, 0]]
```
represents a motif of length 2, whose first position contains either an A or T, each with probability 0.5, and its second position contains either a C or a G, each with probability 0.5.

In [2]:
def gibbs_motif(file_name, length, iterations) :
    """
    Discover motifs using the Gibbs sampling approach.
    
    parameters:
    file_name - fasta file containing the sequences
    length - the length of the motif
    iterations - the number of iterations to run the algorithm

    Returns a motif as a list of lists or two dimensional NumPy array
    """
    
    return [[0.5, 0, 0, 0.5], [0, 0.5,0.5, 0]]

In [1]:
# run your code and comment on the results

## Part 2:  a review of biology terms

Answer the following questions (up to a paragraph for each question):

* Transcription factors have a strong impact on many biological processes.  Explain how!  How many transcription factors are coded by the human genome?  Compare that to the total number of genes.  Wikipedia is a good source ( [article](https://en.wikipedia.org/wiki/Transcription_factor) ), as is the following article:  [The Human Transcription Factors
](https://www.cell.com/cell/fulltext/S0092-8674(18)31257-1).
  

*your answer here*

* What makes the motif finding problem difficult?  How does the biological technique of ChIP-seq assist in that when it comes to determining the binding sites of transcription factors?

*your answer here*

* What is a promoter and enhancer, and how they are relevant to the topic of motif finding?

*your answer here*

* In class we also described an algorithm called *RandomizedMotifSearch*.  Why do you expect Gibbs sampling to perform better?

*your answer here*

### Submission

Submit your assignment as a Jupyter notebook via Canvas.  

### Grading 

Here is what the grading sheet will look like for this assignment. 

```
Grading sheet for assignment 2
- Correctness of your Gibbs sampler, including test on toy data (70 pts)
- Results and analysis (15 pts)
- Biology questions (15 pts)
```
